In [7]:
pip install arxiv

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
pip install wikipedia

  Using cached wikipedia-1.4.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [4]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
wiki.name

'wikipedia'

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()

In [9]:
import logging
logging.basicConfig(level=logging.DEBUG)


In [10]:
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200,).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retriver=vectordb.as_retriever()
retriver

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /ap

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024D63C8B350>, search_kwargs={})

In [11]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriver,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [12]:
retriever_tool.name

'langsmith_search'

In [14]:
pip install arxiv


  Using cached arxiv-2.1.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Using cached sgmllib3k-1.0.0-py3-none-any.whl
Using cached arxiv-2.1.3-py3-none-any.whl (11 kB)
Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)


In [15]:
#Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wraper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wraper)
arxiv.name

'arxiv'

In [16]:
tools=[wiki,arxiv,retriever_tool]

In [17]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Generative_AI\\cvenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000024D571FA200>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorsto

In [18]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama2")
llm

Ollama()

In [19]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.smith.langchain.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (2): api.smith.langchain.com:443
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /info HTTP/1.1" 200 485
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /info HTTP/1.1" 200 485
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /commits/hwchase17/openai-functions-agent/latest HTTP/1.1" 200 1215


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [20]:
#Agents 
from langchain_community.agents import create_